In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time
import gc
import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
'''
临时表一：数据提取与预处理，定义ORDER_table临时表，获取渠道、活动、订单用户的相关信息，查询条件包括订单创建时间在最近7天内，创建时间小于18点，
用户手机号不为空，用户真实姓名不在名单内，活动名称不在名单内，商家ID不在名单内；

临时表二：数据聚合和分类，按照渠道类型和创建日期（1,2）进行分组
计算每个分组中的去重用户数（DAU）
计算每个分组中状态不在（1, 13）的去重用户数（即实际活跃用户数，jj_dau）
计算每个分组中状态在（2, 3, 4, 5, 6, 8, 15）的去重用户数（即特定状态下的活跃用户数，ck_dau）
最后，通过 union all 将每个渠道的数据和总体数据进行合并输出。
'''

sql1 = ''' -- 每日运营数据  
with order_table as (
SELECT om.order_number,om.user_mobile,om.create_time,om.`status`  as status1 
-- ,case om.`status` when 13 then 1 when 1 then 2 when 10 then 3 when 12 then 4 when 2 then 5 when 3 then 6 when 4 then 7 when 8 then 8 else 9 end as status_rank 
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' 
when locate('抖音',pa.name)>0 or om.activity_id in (291,292,293) then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
-- ,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,date(om.create_time) as create_date 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,tmu.sex    -- '性别。示例：0，未知，1、男  2、女',
,tmu.id_card_num 
,om.merchant_id,om.merchant_name

from  db_digua_business.t_order  om 
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
where om.create_time>= DATE_ADD(CURRENT_DATE,INTERVAL -7 day )               -- 近7天数据
and hour(om.create_time)<18
and om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文")
 and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- and pa.name not like '%抖音%' and om.activity_id not in (291,292,293)
and om.merchant_id not in (15,18,21,30,31,33,79,85,99,107,124,171,172)  -- 剔除商家

order by 2,3,5)


(SELECT t1.channel_type ,t1.create_date,count(DISTINCT t1.user_mobile) as  dau -- 去重订单
,count(DISTINCT case when t1.status1 not in (1,13) then t1.user_mobile end) as jj_dau
--  ,count( case when t1.status1 not in (1,13) then t1.user_mobile end) as jj_dau_1
,count(DISTINCT case when t1.status1 in (2,3,4,5,6,8,15) then t1.user_mobile end) as ck_dau

from  order_table  t1 
GROUP BY 1,2) 
union all 
(SELECT '总体' as channel_type ,t1.create_date,count(DISTINCT t1.user_mobile) as  dau -- 去重订单
,count(DISTINCT case when t1.status1 not in (1,13) then t1.user_mobile end) as jj_dau 
--  ,count( case when t1.status1 not in (1,13) then t1.user_mobile end) as jj_dau_1
,count(DISTINCT case when t1.status1 in (2,3,4,5,6,8,15) then t1.user_mobile end) as ck_dau

from  order_table  t1 
GROUP BY 1,2)
;
'''

In [4]:
# 该查询用于统计近16天每日进入信审的订单数量、免审订单数量、非免审订单数量、审核拒绝数量、客户取消数量、无法联系数量、命中出库前风控流强拒数量、出库数量等指标
sql2 = ''' -- 信审每日数据
-- 免审or 非免审 订单 信审 转化
SELECT date(om.create_time) as create_date 
,count(om.id) as 进入信审
,count(case when locate('免人审',tor.decision_result)>0 then 1 end) as  免审订单
,(count(om.id) - count(case when locate('免人审',tor.decision_result)>0 then 1 end))  as  非免审进件

,count(case when locate('审核不通过：',top.total_describes)>0 then 1 end) as  审核拒绝 

,count(case when locate('审核不通过：',top.total_describes)>0 and locate('免人审',tor.decision_result)>0 then 1 end) as 免审拒绝
,count(case when locate('客户申请取消',top.total_describes)>0 then 1 end) as  客户取消
,count(case when locate('用户无法联系',top.total_describes)>0 then 1 end) as  无法联系
,count(case when locate('命中出库前风控流强拒',tor.decision_result)>0 then 1 end) as  命中出库前风控流强拒
,count(case when locate('蚂蚁数控风险等级',top.total_describes)>0 then 1 end) as  命中出库前风控流强拒蚂蚁数控
,count(case when locate('客户申请取消',top.total_describes)>0 and locate('免人审',tor.decision_result)>0 then 1 end) as  免审客户取消
,count(case when locate('用户无法联系',top.total_describes)>0 and locate('免人审',tor.decision_result)>0 then 1 end) as  免审无法联系
,count(case when locate('命中出库前风控流强拒',tor.decision_result)>0 and locate('免人审',tor.decision_result)>0 then 1 end) as  免审命中出库前风控流强拒
,count(case when locate('蚂蚁数控风险等级',top.total_describes)>0 and locate('免人审',tor.decision_result)>0 then 1 end) as  免审命中出库前风控流强拒蚂蚁数控
,count(case when om.status in (2,3,4,5,6,8,15) then 1 end) as 出库
,count(case when om.status in (2,3,4,5,6,8,15) and locate('免人审',tor.decision_result)>0 then 1 end) as 免审出库
,(count(case when om.status in (2,3,4,5,6,8,15) then 1 end) - count(case when om.status in (2,3,4,5,6,8,15) and locate('免人审',tor.decision_result)>0 then 1 end)) as 非免审出库 
,count(case when om.status in (2,3,4,5,6,8,15) and locate('免人审',tor.decision_result)>0 then 1 end) / count(case when locate('免人审',tor.decision_result)>0 then 1 end)  as 免审出库比例
,(count(case when om.status in (2,3,4,5,6,8,15) then 1 end) - count(case when om.status in (2,3,4,5,6,8,15) and locate('免人审',tor.decision_result)>0 then 1 end)) / (count(om.id) - count(case when locate('免人审',tor.decision_result)>0 then 1 end)) as 非免审出库比例

from db_digua_business.t_order  om
inner join  (select  t.order_id  from db_digua_business.t_user_verify_order_record t  GROUP BY 1) tuv  on om.id = tuv.order_id 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes
from db_digua_business.t_order_personnel t   GROUP BY 1 ) top  on om.id = top.order_id 
-- left join (SELECT  tso.order_number,tso.price from db_digua_business.t_service_order tso  where tso.`status` in (2,3,4) ) 
where  om.create_time > DATE_ADD(CURRENT_DATE,INTERVAL -16 day )               -- 近7天数据（16天）             
-- and tuv.order_id is not null 
and replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') not LIKE '%策略241205%'
and replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') not LIKE '%策略241212%'
GROUP BY 1 
order by 1 
;
'''

# my_job1

In [5]:
from apscheduler.schedulers.blocking import BlockingScheduler  
from apscheduler.schedulers.background import BackgroundScheduler 
# 每天18点20分执行一次
def my_job1():  
    
    # path1 = r'\\digua\迪瓜租机\13.运营-信审每日数据/'    # 共享盘   \\digua2\迪瓜租机\13.运营-信审每日数据  192.168.1.4 \\digua\迪瓜租机\13.运营-信审每日数据
    path1 = r'\\digua\迪瓜租机\13.运营-信审每日数据/'    # 共享盘   \\digua2\迪瓜租机\13.运营-信审每日数据
    Today1 = str(datetime.datetime.now().strftime('%Y%m%d%H'))
    print(f"执行定时任务：现在是{Today1}的18点20分") 
    order_df = query(sql1)
    order_df.to_excel(path1 + f'每日运营需求数据_{Today1}.xlsx')
    del order_df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler1 = BackgroundScheduler()

# 添加一个cron任务，每天6点20分执行  
# cron表达式格式：(年，月，日，时，分，秒)中的部分字段可以留空，使用*表示  
# 这里我们只需要指定时和分，其他字段使用*表示每天  
scheduler1.add_job(my_job1, 'cron', hour=18, minute=20)    # 18:20  scheduler1.add_job(my_job1, 'cron', hour=18, minute=20) 

# 启动scheduler  
scheduler1.start()
# my_job1()

In [6]:
# from apscheduler.schedulers.blocking import BlockingScheduler   
# from apscheduler.schedulers.background import BackgroundScheduler 
# import datetime 

# # Today = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
 
# def my_job2():  
#     print("执行定时任务：现在是每日的18点20分") 
#     # path2 = r'\\digua\迪瓜租机\13.运营-信审每日数据/'      # 共享盘  \\digua2\迪瓜租机\13.运营-信审每日数据
#     path2 = r'\\digua\迪瓜租机\13.运营-信审每日数据/'
#     Today2 = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
#     verify_df = query(sql2)
#     verify_df.to_excel(path2 +f'每日信审需求数据_{Today2}.xlsx')
  
# scheduler2 = BackgroundScheduler()
  
# # 添加一个cron任务，每天6点20分执行   
# scheduler2.add_job(my_job2, 'cron', hour=18, minute=25)  # 18:30
# # scheduler2.add_job(my_job2, 'date', run_date='2024-08-09 09:56:00')



# # 启动scheduler  
# scheduler2.start()

In [7]:
# scheduler1.shutdown()
# scheduler2.shutdown()

In [8]:
# 查询主要用于统计最近30天内，不同渠道（付费灯火和搜索渠道）的订单情况，包括去重订单数、进件数和出库数
sql3 = ''' -- 渠道运营需求统计
-- 订单去重统计  
with order_table as (
SELECT om.order_number,om.user_mobile,om.create_time,om.`status`  as status1 
-- ,case om.`status` when 13 then 1 when 1 then 2 when 10 then 3 when 12 then 4 when 2 then 5 when 3 then 6 when 4 then 7 when 8 then 8 else 9 end as status_rank 
,date(om.create_time) as create_date 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,tmu.sex    -- '性别。示例：0，未知，1、男  2、女',
,tmu.id_card_num 
,om.merchant_id,om.merchant_name
from db_digua_business.t_order om 
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
where om.create_time>=DATE_ADD(CURRENT_DATE,INTERVAL -30 day )               -- 近14天数据（30天）
and om.user_mobile is not null 
and (cc.name in ('付费灯火-高频词','付费灯火-蓝海词','付费流量(通过商家数字推广平台，灯火等投放的广告)')
or  cc.name like '%搜索%')
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文")
and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
and COALESCE(pa.name, '未知') not like '%抖音%'

order by 2,3,5) 

SELECT  
ot1.create_date
, case when ot1.channel_name in ('付费灯火-高频词','付费灯火-蓝海词','付费流量(通过商家数字推广平台，灯火等投放的广告)') then ot1.channel_name else '搜索渠道' end as channel_type 
,count(DISTINCT ot1.user_mobile) as  去重订单数
,count(DISTINCT case when ot1.status1 not in (1,13) then ot1.user_mobile end) as 进件
,count(DISTINCT case when ot1.status1 in (2,3,4,5,6,8,15) then ot1.user_mobile end) as 出库

from order_table ot1 
GROUP BY 1 ,2 
order by 1 ,2
;
'''

# my_job3

In [9]:
from apscheduler.schedulers.blocking import BlockingScheduler   
from apscheduler.schedulers.background import BackgroundScheduler 
import datetime 

# Today = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
# 每周一9点半执行一次
def my_job3():  
    print("执行定时任务：现在是周一的9点30分")
    path3 = r'\\digua\迪瓜租机\14.运营数据-搜索/'   # 共享盘  \\digua2\迪瓜租机\13.运营-信审每日数据
    Today3 = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
    qudao_df = query(sql3)
    qudao_df.to_excel(path3 + f'渠道运营需求数据_{Today3}.xlsx')
    del qudao_df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler3 = BackgroundScheduler()

# 添加一个cron任务，当天9点30分执行   
scheduler3.add_job(my_job3, 'cron', day_of_week='0',hour=9, minute=30)  
# scheduler3.add_job(my_job3, 'date', run_date='2024-08-16 10:24:00')

# 启动scheduler  
scheduler3.start()
# my_job3()

In [10]:
# scheduler3.shutdown()

In [11]:
# 提取最近30天内特定条件下的订单信息，包括订单的创建时间、用户手机号、订单描述、风险决策结果等，用于信审需求分析
sql4 = ''' -- 信审需求  每周五 10点
-- 订单备注信息 
with  describe_table as (
SELECT  top.order_id
-- ,top.create_time,top.describes,top.type 
,JSON_ARRAYAGG(top.describes) as  total_describes
from db_digua_business.t_order_personnel top
where top.order_id is not null 
GROUP BY 1 
order by 1) 

select om.create_time,om.user_mobile,date(om.create_time) as create_date,om.order_number,dt.order_id  
,dt.total_describes,tor.decision_result 
from  db_digua_business.t_order  om 
left join describe_table  dt on dt.order_id = om.id    
left join db_digua_business.t_order_risk tor on om.id = tor.order_id 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id 
where om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -30 day )               -- 近30天数据
and om.user_mobile is not null
and dt.order_id is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"                    
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文")
and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
and om.merchant_id not in (15,18,21,30,31,33,79,85,99,107,124)  -- 剔除商家

;
'''


In [12]:
# reson_df = query(sql4)

# reson_df["取消原因"] = reson_df["total_describes"].str.split("客户申请取消：").str[1].str.split('"').str[0].str.strip()
# reson_df["电审拒绝原因"] = reson_df["total_describes"].str.split("审核不通过：").str[1].str.split('"').str[0].str.strip()
# reson_df["无法联系原因"] = reson_df["total_describes"].str.split("用户无法联系：").str[1].str.split('"').str[0].str.strip()
# reson_df['免审'] = np.where(reson_df.decision_result.str.contains(pat='免人审',regex=False),1,0)
# reson_df.head()

In [13]:
# pd.crosstab(reson_df.create_date,reson_df.无法联系原因,margins=True)

# my_job4

In [14]:
from apscheduler.schedulers.blocking import BlockingScheduler   
from apscheduler.schedulers.background import BackgroundScheduler 
import datetime 
# path = r'\\digua\迪瓜租机\15.运营-每周五取消与拒绝原因/'
# Today = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
# 每周五9点40执行一次
def my_job4():  
    
    t_date = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
    print(f"执行定时任务：现在是{t_date}的40") 
    reson_df = query(sql4)
    reson_df.sort_values(by=['create_time','user_mobile'],inplace= True)
    reson_df.drop_duplicates(subset=['user_mobile'],keep='last',inplace=True)
    reson_df["取消原因"] = reson_df["total_describes"].str.split("客户申请取消：").str[1].str.split('"').str[0].str.strip()
    reson_df["电审拒绝原因"] = reson_df["total_describes"].str.split("审核不通过：").str[1].str.split('"').str[0].str.strip()
    reson_df["无法联系原因"] = reson_df["total_describes"].str.split("用户无法联系：").str[1].str.split('"').str[0].str.strip()
    reson_df['免审'] = np.where(reson_df.decision_result.str.contains(pat='免人审',regex=False),1,0)
    

    path_result = r'\\digua\迪瓜租机\15.运营-每周五取消与拒绝原因/'         # 共享盘
    excel_name = f'取消&拒绝原因_{t_date}.xlsx'

    qx_zt_df = pd.crosstab(reson_df['create_date'],reson_df['取消原因'],margins=True)
    with pd.ExcelWriter(path_result + excel_name, engine='xlsxwriter') as writer:  
        qx_zt_df.to_excel(writer, sheet_name='取消原因-总体', startrow=0, index=True) 

    jj_zt_df = pd.crosstab(reson_df['create_date'],reson_df['电审拒绝原因'],margins=True)
    with pd.ExcelWriter(path_result + excel_name, engine='openpyxl',mode='a') as writer:  
        jj_zt_df.to_excel(writer, sheet_name='拒绝原因-总体', startrow=0, index=True) 
    
    jj_ms_df = pd.crosstab(reson_df[reson_df.免审==1]['create_date'],reson_df[reson_df.免审==1]['电审拒绝原因'],margins=True)
    with pd.ExcelWriter(path_result + excel_name, engine='openpyxl',mode='a') as writer:  
        jj_ms_df.to_excel(writer, sheet_name='拒绝原因-免审', startrow=0, index=True) 
    
    qx_ms_df = pd.crosstab(reson_df[reson_df.免审==1]['create_date'],reson_df[reson_df.免审==1]['取消原因'],margins=True)
    with pd.ExcelWriter(path_result + excel_name, engine='openpyxl',mode='a') as writer:  
        qx_ms_df.to_excel(writer, sheet_name='取消原因-免审', startrow=0, index=True)
        
    del qx_zt_df, jj_zt_df, jj_ms_df, qx_ms_df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler4 = BackgroundScheduler()

# 添加一个cron任务，当天9点40执行   
scheduler4.add_job(my_job4, 'cron', day_of_week='4',hour=9, minute=40)  
# scheduler4.add_job(my_job4, 'date', run_date='2024-08-16 17:15:00')

# 启动scheduler  
scheduler4.start()
# my_job4()

In [15]:
# scheduler4.shutdown()

In [16]:
sql_bp = ''' -- 八派信息数据需求（大多同日报）
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
-- ,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money,tod.money,tod.pledge,tod.sku_price_back
-- ,JSON_EXTRACT(tod.sku_attributes, '$[?(@.key=="颜色")].value') AS color_value  
-- ,JSON_UNQUOTE(JSON_EXTRACT(tod.sku_attributes, '$[?(@.key=="内存")].value')) AS memory_value
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租完归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
-- ,top.total_describes,tor.decision_result,om.cancel_reason
-- ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
-- ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
-- ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,case when topay.total_freeze_fund_amount is null then 0 else topay.total_freeze_fund_amount end as total_freeze_fund_amount
,om.buy_service_product,tso.status as service_status 

from db_digua_business.t_order  om 
-- left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
# left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
# on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文")
# and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- and om.merchant_id not in (15,18,21,30,31,33,79,85,99,107,124)  -- 剔除商家
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近15天数据    
-- and date_format(om.create_time,'%Y-%m-%d') >= '2025-09-15'
and     cc.name in ('八派信息')      
-- GROUP BY 1 
-- order by 1 
;
'''

In [17]:
sql_jz = ''' -- 九州信息数据需求
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
-- ,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money,tod.money,tod.pledge,tod.sku_price_back
-- ,JSON_EXTRACT(tod.sku_attributes, '$[?(@.key=="颜色")].value') AS color_value  
-- ,JSON_UNQUOTE(JSON_EXTRACT(tod.sku_attributes, '$[?(@.key=="内存")].value')) AS memory_value
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租完归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
-- ,top.total_describes,tor.decision_result,om.cancel_reason
-- ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
-- ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
-- ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,case when topay.total_freeze_fund_amount is null then 0 else topay.total_freeze_fund_amount end as total_freeze_fund_amount
,om.buy_service_product,tso.status as service_status 

from db_digua_business.t_order  om 
-- left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
# left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
# on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文")
# and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- and om.merchant_id not in (15,18,21,30,31,33,79,85,99,107,124)  -- 剔除商家
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近15天数据    
-- and date_format(om.create_time,'%Y-%m-%d') >= '2025-09-15'
and cc.name in ('九州信息', '派金花', '诚派', '聚量派', '98租超', '一铭', '宜品花', '汇租CPS', '聚优源')      
-- GROUP BY 1 
-- order by 1 
;
'''

In [18]:
def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

In [19]:
# baipai_df = query(sql_bp)
# baipai_df.loc[:,"颜色"] = baipai_df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1) 
# baipai_df.loc[:,"内存"] = baipai_df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
# baipai_df.to_excel(r'E:\myfile\数据提取及分析/bapai_df-1.xlsx')

# my_job5

In [20]:
from apscheduler.schedulers.blocking import BlockingScheduler   
from apscheduler.schedulers.background import BackgroundScheduler 
import datetime 
# path = r'\\digua\迪瓜租机\16.运营-外部渠道数据/'
# Today = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
# 每周一的9点执行
def my_job5():  
    print("执行定时任务：现在是周一的9点") 
    # path_bp = r'\\digua2\迪瓜租机\16.运营-外部渠道数据/'         # 共享盘  192.168.1.4
    path_bp = r'\\digua\迪瓜租机\16.运营-外部渠道数据/'
    Today5 = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
    bapai_df = query(sql_bp)
    bapai_df.loc[:,"颜色"] = bapai_df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1) 
    bapai_df.loc[:,"内存"] = bapai_df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)

    bapai_df.to_excel(path_bp + f'八派信息需求数据_{Today5}.xlsx')
    del bapai_df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler5 = BackgroundScheduler()

# 添加一个cron任务，当天9点执行   
scheduler5.add_job(my_job5, 'cron', day_of_week='0',hour=9, minute=0)  
# scheduler5.add_job(my_job5, 'cron' ,hour=14, minute=0)
# scheduler5.add_job(my_job5, 'date', run_date='2024-08-19 15:44:00')

# 启动scheduler  
scheduler5.start()
# my_job5()
# scheduler5.shutdown()

# my_job_jz

In [21]:
from apscheduler.schedulers.blocking import BlockingScheduler   
from apscheduler.schedulers.background import BackgroundScheduler 
import datetime 
# path = r'\\digua\迪瓜租机\16.运营-外部渠道数据/'
# Today = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
# 每周一的9点20分执行
def my_job_jz():  
    print("执行定时任务：现在是周一的9点20分") 
    # path_bp = r'\\digua2\迪瓜租机\16.运营-外部渠道数据/'         # 共享盘  192.168.1.4
    path_jz = r'\\digua\迪瓜租机\16.运营-外部渠道数据/'
    Today_jz = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
    jiuzhou_df = query(sql_jz)
    jiuzhou_df.loc[:,"颜色"] = jiuzhou_df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1) 
    jiuzhou_df.loc[:,"内存"] = jiuzhou_df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)

    jiuzhou_df.to_excel(path_jz + f'九州信息需求数据_{Today_jz}.xlsx')
    del jiuzhou_df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler5 = BackgroundScheduler()

# 添加一个cron任务，当天9点10分执行   
scheduler5.add_job(my_job_jz, 'cron', day_of_week='0',hour=9, minute=20) 
# scheduler5.add_job(my_job_jz, 'cron', day_of_week='0',hour=14, minute=30) 
# scheduler5.add_job(my_job5, 'cron' ,hour=14, minute=0)
# scheduler5.add_job(my_job5, 'date', run_date='2024-08-19 15:44:00')

# 启动scheduler  
scheduler5.start()
# my_job_jz()

In [22]:
# scheduler5.shutdown()


In [23]:
sql_ck = ''' -- 需求--何薪华  
-- 出库名单 
SELECT om.order_number,om.user_mobile,om.create_time,om.`status`  as status1 
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,tmu.mobile 
from db_digua_business.t_order om  
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
where om.create_time>= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近15天数据
and om.`status` in (2,3,4)
;
'''

In [24]:
# ck_df = query(sql_ck)

# ck_df.to_excel(path_ck  + f'出库信息需求数据_{Today}.xlsx')

# my_job6

In [25]:
from apscheduler.schedulers.blocking import BlockingScheduler   
from apscheduler.schedulers.background import BackgroundScheduler 
import datetime 
# 每周一的9点10分执行
def my_job6():  
    print("执行定时任务：现在是周一的9点10分") 
    #path_ck = r'\\digua2\迪瓜租机\17.出库名单/'            # 共享盘  192.168.1.4
    path_ck = r'\\digua\迪瓜租机\17.出库名单/'
    Today6 = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
    ck_df = query(sql_ck)

    ck_df.to_excel(path_ck  + f'出库信息需求数据_{Today6}.xlsx')
    del ck_df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler6 = BackgroundScheduler()

# 添加一个cron任务，周一的9点10分执行   
scheduler6.add_job(my_job6, 'cron', day_of_week='0',hour=9, minute=10)  
# scheduler6.add_job(my_job6, 'date', run_date='2024-08-13 09:38:00')

# 启动scheduler  
scheduler6.start()
# my_job6()

In [26]:
# scheduler5.shutdown()


In [27]:
# from apscheduler.schedulers.background import BackgroundScheduler
# import time

# # 创建后台调度器
# scheduler = BackgroundScheduler()

# # 定义任务函数
# def job():
#     print("定时任务执行：", time.strftime("%Y-%m-%d %H:%M:%S"))
#     Today = str(datetime.datetime.now().strftime('%Y%m%d%H')) 
#     verify_df = query(sql2)
#     verify_df.to_excel(path+f'每日信审需求数据_{Today}.xlsx')
#     order_df = query(sql1)
#     order_df.to_excel(path+f'每日运营需求数据_{Today}.xlsx')

#     # 添加定时任务，设置任务在2023年7月31日10点30分触发
# scheduler.add_job(job, 'date', run_date='2024-08-12 18:56:00')

# # 启动调度器
# scheduler.start()

# # 主线程等待一段时间后结束
# time.sleep(60)

# # 关闭调度器
# scheduler.shutdown()

# print("主线程结束")

In [28]:
# import smtplib  
# from email.mime.multipart import MIMEMultipart  
# from email.mime.text import MIMEText  
# from email.mime.base import MIMEBase  
# from email import encoders  
  
# def send_email_with_attachment(to_email, filename):  
#     # 发件人邮箱账号  
#     from_email = 'zhangna@diguazu.com'  
#     # SMTP服务器信息  
#     smtp_server = 'smtp.qiye.aliyun.com'  
#     smtp_port = 465  
#     smtp_user = 'zhangna@diguazu.com'  
#     smtp_password = 'zhang@123'  
  
#     # 创建带附件的邮件  
#     msg = MIMEMultipart()  
#     msg['From'] = from_email  
#     msg['To'] = to_email  
#     msg['Subject'] = "每日运营需求数据"  
  
#     # 邮件正文  
#     body = "这是邮件正文内容。"  
#     msg.attach(MIMEText(body, 'plain'))
  
#     # df.to_csv(filename, index=False)
#     # 添加附件  
#     with open(filename, "rb") as attachment:  
#         part = MIMEBase('application', 'octet-stream')  
#         part.set_payload(attachment.read())  
#     encoders.encode_base64(part)  
#     part.add_header('Content-Disposition', f"attachment; filename= {filename}")  
#     msg.attach(part)  
  
#     # 发送邮件  
#     server = smtplib.SMTP_SSL(smtp_server, smtp_port)  
#     # server.starttls()  
#     server.login(smtp_user, smtp_password)  
#     text = msg.as_string()  
#     server.sendmail(from_email, to_email, text)  
#     server.quit()  
  
#     print("邮件发送成功！")

In [29]:
# filename = f'每日运营需求数据_{Today}.csv'
# query(sql1).to_csv(filename, index=False)
# send_email_with_attachment('lairuotong@diguazu.com',filename)

In [30]:
# filename = f'每日信审需求数据_{Today}.xlsx'
# query(sql2).to_excel(filename, index=False)
# send_email_with_attachment('lwl@diguazu.com',filename)